# Summarize mangroves in regions

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[mangrove_cover](https://explorer.sandbox.dea.ga.gov.au/products/mangrove_cover)

## Description
Summarize mangroves area of each class (wood_land, open_forest and closed_forest) in regions.
***

In [4]:
import datacube
from datacube import Datacube
import fiona
from datacube.utils.geometry import CRS, Geometry
from shapely import geometry
from rasterio import features
import pandas as pd
from datacube.utils.dask import start_local_dask
import matplotlib.pyplot as plt

In [ ]:
def shape_list(shapefile):
    with fiona.open(shapefile) as allshapes:
        for shape in allshapes:
            yield shape

In [ ]:
# enable local dask cluster
client = start_local_dask(n_workers=1, threads_per_worker=7, memory_limit='56GB')
client

In [ ]:
shapefile = './shapefile/IBRA_Regions/IBRA_MERGED_coastal.shp'
query_time = ('1987-01-01', '2021-01-01')
dc = Datacube()

In [ ]:
# sum mangroves area over each region
# save the results in csvs
name_list = {0: 'not_observed', 1: 'woodland', 2: 'open_forest', 3: 'closed_forest'}
for shape in shape_list(shapefile):
    print("process", shape['properties'].get('Mang_IBRA', ''))
    region_geom = geometry.shape(shape['geometry'])
    query_poly = Geometry(region_geom.convex_hull, crs=CRS('EPSG:3577'))
    datasets = dc.find_datasets(product='mangrove_cover', time=query_time, geopolygon=query_poly, measurements=['cover'])
    data = dc.load(product='mangrove_cover', datasets=datasets, measurements=['cover'], dask_chunks={'time':1, 'x':4000, 'y':4000})
    geobox = data.geobox 
    mask = features.geometry_mask(
        region_geom,
        out_shape=geobox.shape,
        transform=geobox.affine,
        all_touched=True,
        invert=True,
    )
    results = None
    for i in range(0, 4):
        re = data.cover.where((data.cover==i) & mask).count(axis=[1, 2])
        re = re.load().to_dataframe(name=name_list[i]).drop(columns="spatial_ref")
        if results is None:
            results = re
        else:
            results = results.join(re, on='time')
    (results * (25 ** 2)).to_csv(shape['properties'].get('Mang_IBRA')+'.csv')

In [ ]:
# read results from csvs and do stack plot
for shape in shape_list(shapefile):
    csv_fname = shape['properties'].get('Mang_IBRA', '') + '.csv'
    results = pd.read_csv(csv_fname)
    results['time'] = results.time.astype('datetime64[D]')
    results = results.set_index('time')
    results.index = results.index.year
    ax = (results/1e6).plot.bar(stacked=True, figsize=(10,7), color=['#808080', '#9FFF4C', '#5ECC00', '#3B7F00'], legend=False, width=1.0)
    ax.set_title(shape['properties'].get('Mang_IBRA', ''), fontweight="bold", fontsize=16)
    ax.set_xlabel('Time (Years)')
    ax.set_ylabel('Area (km$^{2}$)')
    plt.savefig(shape['properties'].get('Mang_IBRA', '') + '.png')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** Junly 2021

**Compatible datacube version:** 

In [5]:
print(datacube.__version__)

1.8.4.dev81+g80d466a2


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)

**Tags**: :index:`NCI compatible`, :index:`sandbox compatible`, :index:`mangroves`, :index:`landsat 5`, :index:`landsat 7`, :index:`landsat 8`